In [1]:
import math as m
import numpy as np
from sympy import symbols, Eq, solve
import pandas as pd
import scipy.linalg as la

In [2]:
matrix = np.array([
    [np.pi, -np.e, np.sqrt(2),-np.sqrt(3)],
    [np.pi**2, np.e, -(np.e)**2,3/7],
    [np.sqrt(5), -np.sqrt(6), 1,-np.sqrt(2)],
    [np.pi**3, (np.e)**2, -np.sqrt(7),1/9]
], dtype=float)

# **Gaussian Elimination with Scaled Patrial Pivoting**

In [3]:
def gauss_partPiv_s(matrix,b):
  matrix = matrix.astype(float).copy()
  b = b.astype(float).copy()

  matrix = np.c_[matrix,b]
  n = len(matrix)
  max_vals = np.max(np.abs(matrix), axis=1)

  for i in range(n):
    col_vals = matrix[i:,i]
    rela_pivs = np.abs(col_vals/max_vals[i:])
    ind_max = np.argmax(rela_pivs) + i

    if ind_max != i:
      matrix[[i,ind_max]] = matrix[[ind_max,i]]
      max_vals[[i,ind_max]] = max_vals[[ind_max,i]]

    for j in range(i+1,n):
      matrix[j,i:] -= (matrix[j,i]/matrix[i,i])*matrix[i,i:]

  return matrix

In [4]:
def back_sub(matrix):
    A = matrix[:, :-1]
    b = matrix[:, -1]
    n = len(b)
    x = np.zeros_like(b, dtype=float)

    for i in reversed(range(n)):
        sum_ax = np.dot(A[i, i+1:], x[i+1:])
        x[i] = (b[i] - sum_ax) / A[i, i]

    return x

In [5]:
b = np.array([np.sqrt(11),0,np.pi,np.sqrt(2)])
aug_sol=gauss_partPiv_s(matrix,b)

In [6]:
back_sub(aug_sol)

array([ 0.78839379, -3.1254137 ,  0.1675966 ,  4.55700258])

# **Jacobi Method**

In [7]:
def jacobi():
  x1,x2,x3,x4,x5,x6 = symbols('x1,x2,x3,x4,x5,x6')
  syms = [x1,x2,x3,x4,x5,x6]
  eqs = [
      Eq(4*x1-x2-x4,0),
      Eq(-x1+4*x2-x3-x5,5),
      Eq(-x2+4*x3-x6,0),
      Eq(-x1+4*x4-x5,6),
      Eq(-x2-x4+4*x5-x6,-2),
      Eq(-x3-x5+4*x6,6),
  ]
  solvedXi = [solve(eq,sym)[0]for eq,sym in zip(eqs,syms)]
  x_i0 = [0]*6
  tolerance = 10**(-3)

  perIter = []
  errors = [0]
  while True:
    perIter.append(x_i0)

    sub_dict = dict(zip(syms,x_i0))
    x_i1 = [float(eq.subs(sub_dict).evalf()) for eq in solvedXi]

    error = max(abs(x1-x0) for x0,x1 in zip(x_i0,x_i1))
    errors.append(error)

    if error < tolerance:
      perIter.append(x_i1)
      break
    x_i0 = x_i1[:]
  return perIter,errors

In [8]:
perIter,errors = jacobi()
jacobi_dict={
    "iteration":[i for i in range(len(perIter))],
    "x1":[i[0] for i in perIter],
    "x2":[i[1] for i in perIter],
    "x3":[i[2] for i in perIter],
    "x4":[i[3] for i in perIter],
    "x5":[i[4] for i in perIter],
    "x6":[i[5] for i in perIter],
    "errors":errors
}
jacobi_df = pd.DataFrame(jacobi_dict)
jacobi_df

,iteration,x1,x2,x3,x4,x5,x6,errors
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,0.000000,1.250000,0.000000,1.500000,-0.500000,1.500000,1.500000
2,2,0.687500,1.125000,0.687500,1.375000,0.562500,1.375000,1.062500
3,3,0.625000,1.734375,0.625000,1.812500,0.468750,1.812500,0.609375
4,4,0.886719,1.679688,0.886719,1.773438,0.839844,1.773438,0.371094
5,5,0.863281,1.903320,0.863281,1.931641,0.806641,1.931641,0.223633
6,6,0.958740,1.883301,0.958740,1.917480,0.941650,1.917480,0.135010
7,7,0.950195,1.964783,0.950195,1.975098,0.929565,1.975098,0.081482
8,8,0.984970,1.957489,0.984970,1.969940,0.978745,1.969940,0.049179
9,9,0.981857,1.987171,0.981857,1.990929,0.974342,1.990929,0.029682


# **Gauss-Sidel Method**

In [9]:
def gauss_sidel():
  x1,x2,x3,x4,x5,x6 = symbols('x1,x2,x3,x4,x5,x6')
  syms = [x1,x2,x3,x4,x5,x6]
  eqs = [
      Eq(4*x1-x2-x4,0),
      Eq(-x1+4*x2-x3-x5,5),
      Eq(-x2+4*x3-x6,0),
      Eq(-x1+4*x4-x5,6),
      Eq(-x2-x4+4*x5-x6,-2),
      Eq(-x3-x5+4*x6,6),
  ]
  solvedXi = [solve(eq,sym)[0]for eq,sym in zip(eqs,syms)]
  x_i0 = [0]*6
  tolerance = 10**(-3)

  perIter = [x_i0.copy()]
  errors = [0]

  while True:
    x_i1 = x_i0.copy()
    for i in range(6): # Update Per Iteration
      sub_dict = dict(zip(syms,x_i0))
      x_i1[i] = float(solvedXi[i].subs(sub_dict).evalf())

    error = max(abs(x1-x0) for x0,x1 in zip(x_i1,x_i0))
    perIter.append(x_i1.copy())
    errors.append(error)

    if error < tolerance:
      break
    x_i0 = x_i1.copy()
  return perIter,errors

In [10]:
perIter,errors = gauss_sidel()

In [11]:
gause_sidel_dict={
    "iteration":[i for i in range(len(perIter))],
    "x1":[i[0] for i in perIter],
    "x2":[i[1] for i in perIter],
    "x3":[i[2] for i in perIter],
    "x4":[i[3] for i in perIter],
    "x5":[i[4] for i in perIter],
    "x6":[i[5] for i in perIter],
    "errors":errors
}
gause_sidel_dict_df = pd.DataFrame(gause_sidel_dict)
gause_sidel_dict_df

,iteration,x1,x2,x3,x4,x5,x6,errors
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,0.000000,1.250000,0.000000,1.500000,-0.500000,1.500000,1.500000
2,2,0.687500,1.125000,0.687500,1.375000,0.562500,1.375000,1.062500
3,3,0.625000,1.734375,0.625000,1.812500,0.468750,1.812500,0.609375
4,4,0.886719,1.679688,0.886719,1.773438,0.839844,1.773438,0.371094
5,5,0.863281,1.903320,0.863281,1.931641,0.806641,1.931641,0.223633
6,6,0.958740,1.883301,0.958740,1.917480,0.941650,1.917480,0.135010
7,7,0.950195,1.964783,0.950195,1.975098,0.929565,1.975098,0.081482
8,8,0.984970,1.957489,0.984970,1.969940,0.978745,1.969940,0.049179
9,9,0.981857,1.987171,0.981857,1.990929,0.974342,1.990929,0.029682


# **Successive Over-Relaxation (SOR) Method**

In [12]:
A = np.array([
    [10,5,0,0],
    [5,10,-4,0],
    [0,-4,8,-1],
    [0,0,-1,5],
])
b = np.array([6,25,-11,-11])

def calSOR(A,w):
  D = np.diag(np.diagonal(A))
  L = np.tril(A,-1)
  U = np.triu(A,1)
  return la.inv(D+(w*L)).dot((1-w)*D-(w*U))

def spectral_m(A):
  eigen = la.eigvals(A)
  return max(abs(eigen))

w_val = np.linspace(1.01,2.0,100)
spectral_rad = []

for omega in w_val:
  sor_vet = calSOR(A,omega)
  spectral_rad.append(spectral_m(sor_vet))

opti_w = w_val[np.argmin(np.array(spectral_rad))]
print(f"Optimal Omega Value: {opti_w}")

Optimal Omega Value: 1.16
